In [7]:
# STEP 5: Latent Factor Model 


# Define hyperparameters to tune
svd_param_grid = {
    'n_factors': [50, 100, 150],
    'n_epochs': [20, 30],
    'lr_all': [0.005, 0.01],
    'reg_all': [0.02, 0.1]
}

print("Tuning SVD hyperparameters (this may take several minutes)...")

# Grid search for best SVD parameters
svd_grid = GridSearchCV(SVD, svd_param_grid, measures=['rmse'], cv=5)
svd_grid.fit(data)

# Best parameters and score
print(f"\nBest SVD parameters: {svd_grid.best_params['rmse']}")
print(f"Best SVD RMSE: {svd_grid.best_score['rmse']:.4f}")

# Train final SVD model with best parameters
best_svd = svd_grid.best_estimator['rmse']
best_svd.fit(trainset)

print("SVD Model trained successfully!")


Tuning SVD hyperparameters (this may take several minutes)...

Best SVD parameters: {'n_factors': 50, 'n_epochs': 20, 'lr_all': 0.01, 'reg_all': 0.1}
Best SVD RMSE: 0.9543
SVD Model trained successfully!


In [6]:
# STEP 4: Neighborhood-based Model

# Define hyperparameters to tune
knn_param_grid = {
    'k': [20, 30, 40, 50],
    'sim_options': {
        'name': ['cosine', 'pearson'],
        'user_based': [True, False]  # True = user-user, False = item-item
    }
}


print("Tuning hyperparameters with GridSearchCV (this may take a few minutes)...")

# Grid search for best KNN parameters
knn_grid = GridSearchCV(KNNBasic, knn_param_grid, measures=['rmse'], cv=5)
knn_grid.fit(data)

# Best parameters and score
print(f"\nBest KNN parameters: {knn_grid.best_params['rmse']}")
print(f"Best KNN RMSE: {knn_grid.best_score['rmse']:.4f}")

# Train final KNN model with best parameters
best_knn = knn_grid.best_estimator['rmse']
trainset = data.build_full_trainset()
best_knn.fit(trainset)

print("\nKNN Model trained successfully!")

Tuning hyperparameters with GridSearchCV (this may take a few minutes)...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing th

In [5]:
# STEP 3: Implement TopPop Recommender

# TopPop = recommend items with most high ratings (≥3)
top_pop_items = top_items.head(10)['item_id'].tolist()
print(f"Top-10 popular items: {top_pop_items}")

# For Surprise we need to create a custom algorithm
from surprise import AlgoBase

class TopPop(AlgoBase):
    def __init__(self, top_items_list):
        AlgoBase.__init__(self)
        self.top_items = top_items_list
    
    def fit(self, trainset):
        self.trainset = trainset
        return self
    
    def estimate(self, u, i):
        # If item is in top popular list, predict high rating (5)
        # Otherwise predict low rating (1)
        if self.trainset.to_raw_iid(i) in self.top_items:
            return 5.0
        else:
            return 1.0

# Evaluate TopPop with cross-validation
print("\nEvaluating TopPop with 5-fold CV:")
top_pop_algo = TopPop(top_pop_items)
cv_results = cross_validate(top_pop_algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

print(f"\nTopPop - Average RMSE: {cv_results['test_rmse'].mean():.4f}")
print(f"TopPop - Average MAE: {cv_results['test_mae'].mean():.4f}")

Top-10 popular items: ['B0086VPUHI', 'B00BN5T30E', 'B07YBXFDYN', 'B00BGA9WK2', 'B007CM0K86', 'B00KIWEMIG', 'B07YBWT3PK', 'B07YBXFF99', 'B004HD55VK', 'B014R4KYMS']

Evaluating TopPop with 5-fold CV:
Evaluating RMSE, MAE of algorithm TopPop on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    3.3475  3.3470  3.3477  3.3435  3.3437  3.3459  0.0019  
MAE (testset)     3.0909  3.0848  3.0974  3.0869  3.0807  3.0881  0.0057  
Fit time          0.00    0.00    0.00    0.00    0.00    0.00    0.00    
Test time         0.02    0.00    0.00    0.00    0.00    0.01    0.01    

TopPop - Average RMSE: 3.3459
TopPop - Average MAE: 3.0881


In [3]:
# STEP 2: Prepare data for Surprise library

# put data in correct format
reader = Reader(rating_scale=(1, 5))

# Convert to Surprise dataset
data = Dataset.load_from_df(train_df[['user_id', 'item_id', 'rating']], reader)

In [2]:
# STEP 1: Import libraries and load data

import pandas as pd
import numpy as np
from surprise import Dataset, Reader, SVD, KNNBasic, NormalPredictor
from surprise.model_selection import cross_validate, GridSearchCV
from surprise import accuracy
import warnings
warnings.filterwarnings('ignore')

print(" Loading cleaned data")

# Load the cleaned data from Week 6
train_df = pd.read_parquet('cleaned_train.parquet')
test_df = pd.read_parquet('cleaned_test.parquet')
top_items = pd.read_csv('top_items.csv')

print(f"Training set size: {len(train_df)}")
print(f"Test set size: {len(test_df)}")
print(f"Top items loaded: {len(top_items)}")

 Loading cleaned data
Training set size: 26580
Test set size: 6645
Top items loaded: 932
